# **ARC Technical assessment**

**GOAL:** Evaluate if the data is enough to perform analyses examining customers attributes and transactions to drive business decisions. Formulate a recommendation to the client based on the findings.
Recommendation to client: Data looks OK and there are possibilities for analysis.

In [463]:
# _____ Libraries _____
import pandas as pd
import json

### **<span style="color:purple">LOAD RAW DATA SETS</span>**
Load in the datasets and store them in pandas dataframes.


In [464]:
df_demographics = pd.read_csv('demographics.csv')
df_referral = pd.read_csv('referral.csv')
df_transactions = pd.read_csv('transactions.csv')

In [465]:
# Load the json file and convert it to a dataframe
with open('subscription_prices.json') as f:
    subscription_prices = json.load(f)
df_subscription_prices = pd.DataFrame(subscription_prices)
# df_subscription_prices['YEAR'] = df_subscription_prices.index
# df_subscription_prices = df_subscription_prices.reset_index(drop=True)
df_subscription_prices

,BASIC,PRO,MAX
2020,33,65,99
2021,43,75,109
2022,53,85,119


## **<span style="color:purple">DATASET 1: *Demographics*</span>**

In [466]:
# Number of features
print(df_demographics.info())
# Number of unique values for each feature
df_demographics.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10366 entries, 0 to 10365
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cust_id           10366 non-null  object
 1   customer_gender   10366 non-null  object
 2   age_group         10366 non-null  object
 3   customer_country  10366 non-null  object
dtypes: object(4)
memory usage: 324.1+ KB
None


,cust_id,customer_gender,age_group,customer_country
count,10366,10366,10366,10366
unique,10366,5,5,6
top,1_cust,Other,18-24,Sweden
freq,1,4460,2536,4061


In [467]:
# Preview the first 10 rows of the dataframe
df_demographics.head(10)

,cust_id,customer_gender,age_group,customer_country
0,1_cust,Male,35-44,Denmark
1,2_cust,Other,18-24,Denmark
2,3_cust,Other,25-34,Norway
3,4,Male,18-24,Sweden
4,5_cust,Other,35-44,Sweden
5,6_cust,Other,55-65,Sweden
6,7,Other,25-34,Sweden
7,8_cust,Other,45-54,Sweden
8,9_cust,Female,25-34,Sweden
9,10,Other,25-34,Finland


In [468]:
# Number of nan-values for each feature
df_demographics.isna().sum()

cust_id             0
customer_gender     0
age_group           0
customer_country    0
dtype: int64

### **<span style="color:purple">Demographics: "cust_id"</span>**
Some values in the "cust_id" column contain "_cust" at the end, which is not consistent with the other values in the column. Let's remove these redundant "cust_id".

In [469]:
# Remove "_cust" from the "cust_id" column
df_demographics['cust_id'] = df_demographics['cust_id'].str.replace('_cust', '')

# Preview the first 10 rows of the dataframe to check if the changes were made
df_demographics.head(10)

,cust_id,customer_gender,age_group,customer_country
0,1,Male,35-44,Denmark
1,2,Other,18-24,Denmark
2,3,Other,25-34,Norway
3,4,Male,18-24,Sweden
4,5,Other,35-44,Sweden
5,6,Other,55-65,Sweden
6,7,Other,25-34,Sweden
7,8,Other,45-54,Sweden
8,9,Female,25-34,Sweden
9,10,Other,25-34,Finland


Check if there are any duplicates in the "cust_id" column. Presumably, there should be none as each customer should have a unique ID.

In [470]:
print('Are there any duplicates in the "cust_id" column? Answer:', df_demographics['cust_id'].duplicated().any())

Are there any duplicates in the "cust_id" column? Answer: False


### **<span style="color:purple">Demographics: "customer_gender"</span>**
There are 5 different values present in the "customer_gender" column while there should be 3: "Female", "Male", and "Other".
Let's preview all values.

In [471]:
# List all 5 unique values for the column
df_demographics.customer_gender.value_counts()

Other     4460
Female    3009
Male      2754
m           88
f           55
Name: customer_gender, dtype: int64

The "m" and "f" values are presumably misspellings so let's correct them.

In [472]:
def handle_misspelled_values(df, col, word_spelling_dict):
    '''
    Loop through datapoints with misspelled values and replace values with the correct spelling.
    :param df: dataframe
    :param col: column name
    :param word_spelling_dict: dictionary with misspelled and correct values
    :return: dataframe with corrected values
    '''

    # Loop over pair of wrong and correct spelled labels
    for wrong_spelling, correct_spelling in word_spelling_dict.items():
        #print('Wrong spelling: ', wrong_spelling, '| Correct spelling: ', correct_spelling)
        index_wrong_spelling = df.index[df[col] == wrong_spelling].tolist() # Get index of wrong spelled values

        for index in index_wrong_spelling:
            df.loc[index, col] = correct_spelling # Replace wrong spelled values with correct spelling
    return df

In [473]:
customer_gender_spellings = {'f': 'Female', 'm': 'Male'}
full_train_df = handle_misspelled_values(df_demographics, 'customer_gender', customer_gender_spellings)
# Check if the misspelled values were corrected
df_demographics.customer_gender.value_counts()

Other     4460
Female    3064
Male      2842
Name: customer_gender, dtype: int64

### **<span style="color:purple">Demographics: "age_group"</span>**


In [474]:
# List all unique values for the column
df_demographics.age_group.value_counts()

18-24    2536
25-34    2117
45-54    1945
55-65    1892
35-44    1876
Name: age_group, dtype: int64

Everything looks good here, i.a. no outliers, misspellings, NaN-values etc., so we can move on to the next column.

### **<span style="color:purple">Demographics: "customer_country"</span>**

In [475]:
df_demographics.customer_country.value_counts()

Sweden     4061
Denmark    2153
Norway     2025
Finland    1975
Sverige      83
Norge        69
Name: customer_country, dtype: int64

There is a mix of country names in English and in Swedish. Let's convert all country names to English as it is the language used the most throughout the datasets.
Let's use the defined function "def handle_misspelled_values()" from above to convert the names.

In [476]:
# Create a dictionary with Swedish and English country names
country_spellings = {'Sverige': 'Sweden', 'Norge': 'Norway'}
handle_misspelled_values(df_demographics, 'customer_country', country_spellings)
# Check if the misspelled values were corrected
df_demographics.customer_country.value_counts()

Sweden     4144
Denmark    2153
Norway     2094
Finland    1975
Name: customer_country, dtype: int64

We have now checked and corrected values in the "demographics" dataset. Let's move on to the next dataset.

## **<span style="color:purple">DATASET 2: *Transactions*</span>**


In [477]:
# Number of features
print(df_transactions.info())
# Number of unique values for each feature
df_transactions.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19141 entries, 0 to 19140
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cust_id            19141 non-null  int64 
 1   transaction_type   19141 non-null  object
 2   transaction_date   19141 non-null  object
 3   subscription_type  19141 non-null  object
dtypes: int64(1), object(3)
memory usage: 598.3+ KB
None


,cust_id,transaction_type,transaction_date,subscription_type
count,19141.000000,19141,19141,19141
unique,NaN,5,36,5
top,NaN,initial,2022-11-01,BASIC
freq,NaN,10366,764,7058
mean,5179.489003,NaN,NaN,NaN
std,2983.089553,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN
25%,2604.000000,NaN,NaN,NaN
50%,5154.000000,NaN,NaN,NaN
75%,7766.000000,NaN,NaN,NaN


In [478]:
# Preview the first 10 rows of the dataframe
df_transactions.head(10)

,cust_id,transaction_type,transaction_date,subscription_type
0,1,initial,2020-09-01,BASIC
1,2,initial,2022-03-01,BASIC
2,3,initial,2020-10-01,MAX
3,4,initial,2022-05-01,PRO
4,5,initial,2022-08-01,BASIC
5,6,initial,2022-06-01,MAX
6,7,initial,2022-02-01,MAX
7,8,initial,2021-01-01,BASIC
8,9,initial,2020-08-01,BASIC
9,10,initial,2022-09-01,MAX


In [479]:
# Number of nan-values for each feature
df_transactions.isna().sum()

cust_id              0
transaction_type     0
transaction_date     0
subscription_type    0
dtype: int64

### **<span style="color:purple">Transactions: "cust_id"</span>**
Everything looks good here, i.a. no outliers, misspellings, NaN-values etc., so we can move on to the next column.
Duplicates are not a problem here as each customer can have multiple transactions.

### **<span style="color:purple">Transactions: "transaction_type"</span>**


In [480]:
# List all unique values for the column
df_transactions.transaction_type.value_counts()

initial      10366
UPGRADE       3692
REDUCTION     2525
CHURN         2511
                47
Name: transaction_type, dtype: int64

In [481]:
# Convert all values to uppercase
df_transactions.transaction_type = df_transactions.transaction_type.str.upper()

There are 47 datapoints which have an empty string as a value. Let's preview thsese.

In [482]:
# List all rows with "transaction_type" = ""
df_transactions[df_transactions.transaction_type == ' ']

,cust_id,transaction_type,transaction_date,subscription_type
10390,95,,2021-06-01,MAX
10447,262,,2021-04-01,PRO
10463,301,,2022-11-01,MAX
10570,596,,2022-10-01,MAX
10639,786,,2021-08-01,MAX
10758,1177,,2022-08-01,MAX
10759,1178,,2021-12-01,MAX
11185,2332,,2020-12-01,MAX
11298,2655,,2022-09-01,MAX
11477,3184,,2022-11-01,MAX


Let's check if a customer with "transaction_type" = "", e.g. "cust_id" = 95, has any other transactions.

In [483]:
df_transactions[df_transactions['cust_id'] == 95]

,cust_id,transaction_type,transaction_date,subscription_type
94,95,INITIAL,2020-11-01,BASIC
10390,95,,2021-06-01,MAX


As we can see, the customer with "cust_id" = 95 has other transactions, so we can assume that the value (an empty string) is faulty and should instead be replaced by "UPGRADE". My reasoning here is that it is evident that the customer in question signed up for the basic-subscription in 2020, and later upgraded to the max-subscription in 2021 – the "transaction_type" should therefore be set to "UPGRADE".
To fix this, my approach would be to go through all the rows where "transaction_type" = "" and replace the empty string with either "UPGRADE"  or "REDUCTION", if the customer appears earlier (time-wise according to the "transaction_date" value) in the transactions-dataset. I assume that this would be a relevant approach as that there could be more rows, like "cust_id"=95, where the empty string in the "transaction_type" column is simply a missing value. Otherwise, the row could simply be a faulty datapoint and thus be dropped.
Given the suggested time limit of about 1h for the assignment, I will wait with this as I reckon it would take a while to implement. Thus, I will leave these rows as they are for now.

### **<span style="color:purple">Transactions: "transaction_date"</span>**

In [484]:
# List all unique values for the column sorted by descending order, i.e. most recent first
df_transactions.transaction_date.value_counts()


2022-11-01    764
2022-10-01    763
2022-06-01    755
2022-08-01    739
2022-07-01    737
2022-03-01    707
2022-04-01    705
2022-09-01    702
2022-01-01    658
2022-02-01    655
2022-05-01    644
2021-12-01    639
2021-11-01    636
2021-10-01    616
2021-09-01    580
2021-08-01    577
2021-06-01    568
2021-05-01    567
2021-07-01    553
2021-04-01    503
2021-02-01    495
2021-03-01    471
2021-01-01    471
2020-12-01    461
2020-11-01    423
2020-09-01    423
2020-10-01    399
2020-07-01    389
2020-08-01    383
2020-06-01    339
2020-04-01    330
2020-05-01    310
2020-01-01    307
2020-02-01    302
2020-03-01    295
2022-12-01    275
Name: transaction_date, dtype: int64

Everything looks good here, i.a. no outliers, misspellings, NaN-values etc., so we can move on to the next column.
Duplicates are not a problem here as each date can occur multiple times as there are multiple customers/transactions.

### **<span style="color:purple">Transactions: "subscription_type"</span>**

In [485]:
df_transactions.subscription_type.value_counts()

BASIC    7058
PRO      6452
MAX      5469
max        83
pro        79
Name: subscription_type, dtype: int64

In [486]:
# Convert all values to uppercase
df_transactions.subscription_type = df_transactions.subscription_type.str.upper()

Everything looks good here, i.a. no outliers, misspellings, NaN-values etc., so we can move on to the next dataset.

## **<span style="color:purple">DATASET 3: *referral*</span>**

In [487]:
# Number of features
print(df_referral.info())
# Number of unique values for each feature
df_referral.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19141 entries, 0 to 19140
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             19141 non-null  object
 1   referral_type  10996 non-null  object
dtypes: object(2)
memory usage: 299.2+ KB
None


,id,referral_type
count,19141,10996
unique,10366,7
top,2184,facebook
freq,6,3023


In [488]:
# Preview the first 10 rows of the dataframe
df_referral.head(10)

,id,referral_type
0,1,NaN
1,2,NaN
2,3,facebook
3,4,Display
4,5,NaN
5,6,facebook
6,7,facebook
7,8,facebook
8,9,facebook
9,10,NaN


In [489]:
# Number of nan-values for each feature
df_referral.isna().sum()

id                  0
referral_type    8145
dtype: int64

### **<span style="color:purple">Referral: "id"</span>**
Some values in the "id" column contain "_ref" at the end, which is not consistent with the other values in the column. Let's remove these redundant "_ref".


In [490]:
# Rename the column to "cust_id" to avoid confusion
df_referral = df_referral.rename(columns={'id': 'cust_id'})

In [491]:
# Remove "_ref" from the values in the "id" column
df_referral['cust_id'] = df_referral['cust_id'].str.replace('_ref', '')

We have in total 19 141 datapoints in the "referral" dataset.
It is not clear what the "id" column represents, but let's assume that it corresponds to the customer ID in the other two datasets. Let's check if there are any duplicates in the "id" column.

In [492]:
print('Are there any duplicates in the "cust_id" column? Answer:', df_referral['cust_id'].duplicated().any())

Are there any duplicates in the "cust_id" column? Answer: True


It is clear that there are duplicates in the "id" column but does this mean that there are duplicates of the full rows as well? Let's check by dropping all duplicates and see if the new number of rows matches the number of unique values in the "id" column (10366).

In [493]:
df_referral = df_referral.drop_duplicates()
print(df_referral.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10366 entries, 0 to 10365
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cust_id        10366 non-null  object
 1   referral_type  5968 non-null   object
dtypes: object(2)
memory usage: 243.0+ KB
None


### **<span style="color:purple">Referral: "referral_type"</span>**


In [494]:
# List nan-values for the column
# print(df_referral[df_referral['referral_type'].isna()])
print('Number of NaN-values', df_referral['referral_type'].isna().sum())

# List all unique values for the column
df_referral.referral_type.value_counts()

Number of NaN-values 4398


facebook          1689
Organic Search    1248
Paid Search       1058
Google Ads        1033
TV                 388
Display            340
Bing               212
Name: referral_type, dtype: int64

There are a noteworthy amount of NaN-values in the "referral_type" column which I would ask the client about as it is not clear if these are simply missing values or if they represent the customer not being referred to the service.

*Merge processed datasets into one csv-file*

### **<span style="color:purple">MERGE DATASETS</span>**

In [495]:
# Merge the demographics and referral datasets on the "id" column
df_complete = pd.merge(df_demographics, df_referral, on='cust_id', how='left')
# Preview the first 5 rows of the dataframe
df_complete.head()

,cust_id,customer_gender,age_group,customer_country,referral_type
0,1,Male,35-44,Denmark,NaN
1,2,Other,18-24,Denmark,NaN
2,3,Other,25-34,Norway,facebook
3,4,Male,18-24,Sweden,Display
4,5,Other,35-44,Sweden,NaN


In [496]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10366 entries, 0 to 10365
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   cust_id           10366 non-null  object
 1   customer_gender   10366 non-null  object
 2   age_group         10366 non-null  object
 3   customer_country  10366 non-null  object
 4   referral_type     5968 non-null   object
dtypes: object(5)
memory usage: 485.9+ KB


In [497]:
# Assign correct types to the columns
df_complete['cust_id'] = df_complete['cust_id'].astype('int64')

In [498]:
# Merge the complete dataset with the transactions dataset on the "cust_id" column
df_complete = pd.merge(df_complete, df_transactions, on='cust_id', how='left')
# Preview the first 5 rows of the dataframe
df_complete.head()

,cust_id,customer_gender,age_group,customer_country,referral_type,transaction_type,transaction_date,subscription_type
0,1,Male,35-44,Denmark,NaN,INITIAL,2020-09-01,BASIC
1,2,Other,18-24,Denmark,NaN,INITIAL,2022-03-01,BASIC
2,3,Other,25-34,Norway,facebook,INITIAL,2020-10-01,MAX
3,3,Other,25-34,Norway,facebook,REDUCTION,2022-02-01,BASIC
4,4,Male,18-24,Sweden,Display,INITIAL,2022-05-01,PRO


In [499]:
# Add another column with the year of the transaction
df_complete['transaction_date'] =  pd.to_datetime(df_complete['transaction_date'])
df_complete['transaction_year'] = df_complete['transaction_date'].dt.year

In [500]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19141 entries, 0 to 19140
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   cust_id            19141 non-null  int64         
 1   customer_gender    19141 non-null  object        
 2   age_group          19141 non-null  object        
 3   customer_country   19141 non-null  object        
 4   referral_type      10996 non-null  object        
 5   transaction_type   19141 non-null  object        
 6   transaction_date   19141 non-null  datetime64[ns]
 7   subscription_type  19141 non-null  object        
 8   transaction_year   19141 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 1.5+ MB


In [501]:
def get_subscription_price(subscription_type, transaction_year):
    # Get the price for the subscription type at the time of the transaction
    str_subscription_type = str(subscription_type)
    str_transaction_year = str(transaction_year)
    price = df_subscription_prices.at[str_transaction_year, str_subscription_type]
    return price

for index, row in df_complete.iterrows():
    # Get the price for the subscription type at the time of the transaction
    price = get_subscription_price(row['subscription_type'], row['transaction_year'])
    # Set the price for the subscription type at the time of the transaction
    df_complete.loc[index, 'subscription_price'] = price


In [502]:
# Save the complete dataset to a csv-file
df_complete.to_csv('complete_dataset-FINAL.csv', index=False)